# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [11]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "employee_id",
      "tipo": "int"
    },
    {
      "nombre": "employee_name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "employee_productivity",
  "fields": [
    {
      "nombre": "employee_id",
      "type": "int"
    },
    {
      "name": "productivity",
      "type": "float"
    },
    {
      "name": "ftpr",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "customer_satisfaction",
  "fields": [
    {
      "name": "employee_id",
      "type": "int"
    },
    {
      "name": "css",
      "type": "float"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [12]:
context = [ {'role':'system', 'content':"""
    create table employees (
    ID_Usr INT PRIMARY KEY,
    emp_name VARCHAR(100),
    department VARCHAR(50),
    project VARCHAR(100));
    /* 3 example rows
    select * from employees limit 3
    ID_Usr    emp_name       department      project
    1344      John Smith     IT              Cloud Migration
    1265      Sarah Johnson  Finance         Budget Analysis
    1289      David Lee      HR              Recruitment Drive
    */

    create table employee_productivity (
    ID_Usr INT,
    emp_name VARCHAR(100),
    project VARCHAR(100),
    productivity FLOAT,
    ftpr FLOAT,
    foreign key (ID_Usr) references employees(ID_Usr));

    /* 3 example rows
    select * from employee_productivity limit 3
    ID_Usr    emp_name       project             productivity   ftpr
    1344      John Smith     Cloud Migration     88.5            100
    1265      Sarah Johnson  Budget Analysis     92.0             98
    1289      David Lee      Recruitment Drive   79.4             72
    */

    create table customer_satisfaction (
    ID_Usr INT,
    css FLOAT,
    foreign key (ID_Usr) references employees(ID_Usr));

    /* 3 example rows
    select * from customer_satisfaction limit 3
    ID_Usr    css
    1344      4.5
    1265      4.8
    1289      4.1
    */

"""} ]



In [13]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
Question 1: query to fetch the top 3 employees with the highest productivity along with their projects
select 
    ep.ID_Usr,
    ep.emp_name,
    ep.project,
    ep.productivity
from employee_productivity ep
order by ep.productivity desc
limit 3;

Question 2: write a query to get employees having productivity between 70 to 95 and having ftpr 100 and having css more than 4

select 
    e.ID_Usr,
    e.emp_name,
    e.department,
    ep.project,
    ep.productivity,
    ep.ftpr,
    cs.css
from employees e
join employee_productivity ep on e.ID_Usr = ep.ID_Usr
join customer_satisfaction cs on e.ID_Usr = cs.ID_Usr
where ep.productivity between 70 and 95
  and ep.ftpr = 100
  and cs.css > 4;


"""
})

In [14]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [ ]:
# example 1
#new
context_user = context.copy()
print(return_CCRMSQL(""" Calculate a final score which is the sum of productivity, ftpr and css. Create a column called final score and store the sum from highest to lowest. Dont use alter or update. Do it in query""", context_user))

```sql
select 
    e.ID_Usr,
    e.emp_name,
    e.department,
    ep.project,
    ep.productivity,
    ep.ftpr,
    cs.css,
    ep.productivity + ep.ftpr + cs.css as final_score
from employees e
join employee_productivity ep on e.ID_Usr = ep.ID_Usr
join customer_satisfaction cs on e.ID_Usr = cs.ID_Usr
order by final_score desc;
```


In [23]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Calculate a final score which is the sum of productivity, ftpr and css. Create a column called final score and store the sum from highest to lowest. Dont use alter or update. Do it in query", old_context_user))

This is your SQL:
```sql
SELECT employee_id, (productivity + ftpr + css) AS final_score
FROM employee_productivity
JOIN customer_satisfaction USING (employee_id)
ORDER BY final_score DESC;
```

This query calculates the final score by summing the productivity, ftpr, and css values for each employee, then orders the results from highest to lowest final score.


In [24]:
#new
print(return_CCRMSQL("Get the employees having more than 3 projects and calculate their final score by summing all their productivity , taking average of ftpr and css to create one final score", context_user))

To achieve this, we can use a subquery to calculate the total productivity for employees with more than 3 projects, then calculate the average of ftpr and css for those employees, and finally sum all these values to create the final score. Here's the SQL query for this:

```sql
WITH EmployeeProjects AS (
    SELECT 
        e.ID_Usr,
        COUNT(ep.project) AS num_projects,
        SUM(ep.productivity) AS total_productivity
    FROM employees e
    JOIN employee_productivity ep ON e.ID_Usr = ep.ID_Usr
    GROUP BY e.ID_Usr
    HAVING num_projects > 3
)

SELECT 
    e.ID_Usr,
    e.emp_name,
    SUM(ep.productivity) + AVG(ep.ftpr) + AVG(cs.css) AS final_score
FROM employees e
JOIN employee_productivity ep ON e.ID_Usr = ep.ID_Usr
JOIN customer_satisfaction cs ON e.ID_Usr = cs.ID_Usr
JOIN EmployeeProjects epj ON e.ID_Usr = epj.ID_Usr
GROUP BY e.ID_Usr
```

In this query:
1. The `EmployeeProjects` common table expression (CTE) calculates the total productivity for employees with more tha

In [21]:
#old
print(return_CCRMSQL("Get the employees having more than 3 projects and calculate their final score by summing all their productivity , taking average of ftpr and css to create one final score", old_context_user))

This is your SQL:
```sql
SELECT e.employee_id, e.employee_name, SUM(ep.productivity) + AVG(ep.ftpr) + AVG(cs.css) AS final_score
FROM employees e
JOIN employee_productivity ep ON e.employee_id = ep.employee_id
JOIN customer_satisfaction cs ON e.employee_id = cs.employee_id
GROUP BY e.employee_id, e.employee_name
HAVING COUNT(ep.employee_id) > 3;
```

This SQL query retrieves the employees who have more than 3 projects by joining the employees table with the employee_productivity and customer_satisfaction tables. It then calculates the final score by summing all their productivity, averaging the ftpr, and averaging the css.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

- Case where it didnot work well: Yes there was a case where i gave the prompt and the GPT altered and updated the values on the table (example 1)
- so i had to extend the prompt

what did i learn:
- making the sql query GPT to stay on topic and provide queries related to the table data.
- cpt 3.5 able to perform complex sql tasks like partioning queries, sub queries
- difference between prompt style provides efficient optimized queries (new style gives better queries)